In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lists_file import dumpJoblib, loadJoblib

data = loadJoblib('unique_textdata.joblib')
label = loadJoblib('label_file.joblib')

In [3]:
import pandas as pd

df = pd.read_csv('covid19.csv')

new_label = []
new_text = []
    
for key in list(loadJoblib('unique_textdata.joblib').keys()):
    ass = df.iloc[key]['assessment_option_id']
    txt = data[key]
    if ass != 66 and ass !=68:
        new_label.append(ass)
        new_text.append(txt)


In [4]:
print('63: 　　　　一般事実　 {}個'.format(label.count(63)))
print('64: 　　　　個人事実 　{}個'.format(label.count(64)))
print('65: 　　　意見・感想 　{}個'.format(label.count(65)))
print('66: 　　　　判定困難 　{}個'.format(label.count(66)))
print('67: 　　　　関係なし 　{}個'.format(label.count(67)))
print('68: 事実だが判定困難 　{}個'.format(label.count(68)))

63: 　　　　一般事実　 9825個
64: 　　　　個人事実 　2986個
65: 　　　意見・感想 　21862個
66: 　　　　判定困難 　19個
67: 　　　　関係なし 　2841個
68: 事実だが判定困難 　10個


In [5]:
print('63: 　　　　一般事実　 {}個'.format(new_label.count(63)))
print('64: 　　　　個人事実 　{}個'.format(new_label.count(64)))
print('65: 　　　意見・感想 　{}個'.format(new_label.count(65)))
print('66: 　　　　判定困難 　{}個'.format(new_label.count(66)))
print('67: 　　　　関係なし 　{}個'.format(new_label.count(67)))
print('68: 事実だが判定困難 　{}個'.format(new_label.count(68)))

63: 　　　　一般事実　 9825個
64: 　　　　個人事実 　2986個
65: 　　　意見・感想 　21862個
66: 　　　　判定困難 　0個
67: 　　　　関係なし 　2841個
68: 事実だが判定困難 　0個


In [6]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.2.7

In [7]:
import random 
import glob
import json
from tqdm import tqdm
import unicodedata

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from transformers import BertTokenizer, BertModel

from lists_file import dumpJoblib, loadJoblib
from multi_classification import BertForSequenceClassificationMultiLabel

MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [9]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
bert_scml = BertForSequenceClassificationMultiLabel(
    MODEL_NAME, num_labels=4
) 
bert_scml = bert_scml.to(device=device)

In [10]:
labels = []
for emotion in new_label:
    if emotion == 63:
        labels.append([1, 0, 0, 0])
    elif emotion == 64:
        labels.append([0, 1, 0, 0])
    elif emotion == 65:
        labels.append([0, 0, 1, 0])
    else:
        labels.append([0, 0, 0, 1])

In [11]:
max_length = 152
dataset_for_loader = []

for text, label in zip(new_text, labels):
    encoding = tokenizer(
      text, 
      max_length=max_length,
      padding='max_length',
      truncation=True
    )
    encoding['labels'] = label
    encoding = {k: torch.tensor(v).to(device=device) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)
    

random.shuffle(dataset_for_loader)
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train]
dataset_val = dataset_for_loader[n_train:n_train+n_val]
dataset_test = dataset_for_loader[n_train+n_val:]

In [12]:
dataloader_train = DataLoader(
    dataset_train, batch_size=16, shuffle=True
)
dataloader_val = DataLoader(
    dataset_val, batch_size=16
)
dataloader_test = DataLoader(
    dataset_test, batch_size=16
)

dataloader_val

In [13]:
from pytorch_lightning_multi import BertForSequenceClassificationMultiLabel_pl

checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='part/model/'
)

#学習方法指定
trainer = pl.Trainer(
    gpus=1,
    max_epochs=1,
    callbacks=[checkpoint]
)

model = BertForSequenceClassificationMultiLabel_pl(
    MODEL_NAME, num_labels=4, lr=1e-5
)

trainer.fit(model, dataloader_train, dataloader_val)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                                    | Params
----------------------------------------------------------------------
0 | bert_scml | BertForSequenceClassificationMultiLabel | 110 M 
----------------------------------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
442.482   Total estimated model params size (MB)
/home/takakiyuto/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

/home/takakiyuto/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [14]:
best_model_path = checkpoint.best_model_path
print(f'validation data loss:{checkpoint.best_model_score}')

validation data loss:0.229273721575737


In [15]:
%load_ext tensorboard
%tensorboard --logdir ./

In [16]:
test = trainer.test(test_dataloaders=dataloader_test, ckpt_path=best_model_path)
print(f'Accuracy: {test[0]["accuracy"]:.4f}')

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/takakiyuto/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'accuracy': 0.7822494506835938}
--------------------------------------------------------------------------------
Accuracy: 0.7822
